Fetching filename from trigger body

In [0]:
filename = dbutils.widgets.get("filename")
print(filename)


Databricks Scope and Azure key vault secrete key :-

In [0]:
storageScope = "DatabricksScope"        
storageKey = "storagesecrete"   
display(storageScope)
display(storageKey)

storagePassword=dbutils.secrets.get(scope=storageScope, key=storageKey) 


**Mount Point creation ** 

In [0]:
alreadyMounted = False
for x in dbutils.fs.mounts():
    if x.mountPoint ==('/mnt/sales'):
     alreadyMounted = True
     break
else:
  alreadyMounted = False
  print("Mounting not done")

In [0]:
if not alreadyMounted:
    dbutils.fs.mount (                                                                                         
    source = "wasbs://sales@cpsalesstorage.blob.core.windows.net",                                      
    mount_point = "/mnt/sales",                                                                           
    extra_configs = {"fs.azure.account.key.cpsalesstorage.blob.core.windows.net": storagePassword}
    )
    alreadyMounted = True
    
    print("Mounting done successfully")
else:   
    print("Already Mounted")

In [0]:
Mount = dbutils.fs.ls("/mnt/sales/")
for f in Mount:
    print(f)

%md
**1st Validation Checks (No Duplicate value of order_id in file) :-**

In [0]:
orderDf = spark.read.csv('/mnt/sales/landing/{}'.format(filename),inferSchema = True , header = True)   
display(orderDf)

count_orderDf=orderDf.count()                                                             
print(count_orderDf)                                                                       

distinct_count_orderDf=orderDf.select('order_id').distinct().count()                     
print(distinct_count_orderDf)                                                             



In [0]:

if count_orderDf !=  distinct_count_orderDf:                                         
    errorFlg =True                             
   
    if errorFlg:
        dbutils.fs.mv('/mnt/sales/landing/{}'.format('filename'), '/mnt/sales/discarded')
        dbutils.notebook.exit('{errorFlg:"True", Error Message:"Duplicate records in orders.csv"}')
else:                                                  
    orderDf.createOrReplaceTempView("orders")         
    print('{OrderTableCreated:"True", Error Message:"No Duplicate records in orders.csv"}')

**2nd Validation Checks (Checking order_status is valid or not) :-**


In [0]:
dbServer ='cpdatabase'
dbPort = '1433'
dbName = 'cpdatabase'
dbUser = 'cpproject@cpdatabase'
dbPassword = 'SQLcredentials'
sqlScope = 'DatabricksScope'


url = "jdbc:sqlserver://{}.database.windows.net:{};database={};user={};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;".format( dbServer, dbPort, dbName, dbUser)

dbPassword= dbutils.secrets.get(scope = sqlScope,key='SQLcredentials')

# Define the connection properties
connection_properties = {
    
    "password": dbPassword,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


validStatusDf = spark.read.jdbc(
    url=url,
    table='dbo.valid_order_status',
    properties=connection_properties
)


In [0]:
validStatusDf.createOrReplaceTempView("valid_status")

invalidRowsDF = spark.sql('select * from orders where order_status not in (select * from valid_status )')
count=invalidRowsDF.count()
print(count)


In [0]:

if invalidRowsDF.count() > 0:
    errorFlg =True
    if errorFlg:
        dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename), '/mnt/sales/discarded')  
        dbutils.notebook.exit('{errorFlg:"True", Error Message:"Invalid order_status found in orders.csv"}')
else:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename), '/mnt/sales/staging')
    print("All Validations successfull file moved to staging area")

**Preparing Final Business solution (Orders placed by each customers & Amount spent by each cusomer) :-**

In [0]:
orderItemsDF = spark.read.csv('/mnt/sales/order_items/order_items.csv', header=True, inferSchema=True)

orderItemsDF.createOrReplaceTempView("order_items")

In [0]:

# Create a DataFrame using read.jdbc with the connection properties
customerDF = spark.read.jdbc(
    url=url,
    table='customers',
    properties=connection_properties
)
customerDF.createOrReplaceTempView("customers")


In [0]:
stagingordersDF= spark.read.csv('/mnt/sales/staging/{}'.format(filename), header=True, inferSchema=True)

stagingordersDF.createOrReplaceTempView("stagingorders")


In [0]:
Final_result_DF = spark.sql("select customers.customer_id, customers.customer_fname, customers.customer_lname, customers.customer_city, customers.customer_state,customers.customer_zipcode,count(order_id) as num_orders_placed, round(sum(order_items.order_item_subtotal),2) as total_amount_spent from customers, stagingorders,order_items where customers.customer_id = stagingorders.customer_id and stagingorders.order_id = order_items.order_item_order_id group by customers.customer_id, customers.customer_fname, customers.customer_lname, customers.customer_city, customers.customer_state, customers.customer_zipcode order by num_orders_placed desc")

**Required Business solution staged in Azure SQL Database**

In [0]:

# Create a DataFrame using read.jdbc with the connection properties
Final_result_DF.write.jdbc(
    url=url,
    table='dbo.sales_reporting',
    mode='overwrite',
    properties=connection_properties
)


In [0]:
print('Final Business Solution exported in Database')